In [2]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.5 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os

DATA_YAML = "/content/drive/MyDrive/hornet_wasp_bee_dataset/data.yaml"
SAVE_DIR  = "/content/trained_models" # folder to save all models

os.makedirs(SAVE_DIR, exist_ok=True)

# Model + resolution combos
RUNS = [
    ("yolo11m.pt", "yolo11m_640", 640),
    ("yolo11s.pt", "yolo11s_640", 640),
    ("yolo11l.pt", "yolo11l_640", 640),
    ("yolo11x.pt", "yolo11x_640", 640),
]

results_summary = []


# ==============================================
# TRAIN + SAVE + EVAL LOOP
# ==============================================
for weights, run_name, imgsz in RUNS:

    print(f"\n===============================")
    print(f"TRAINING: {run_name}  (imgsz={imgsz})")
    print(f"===============================\n")

    model = YOLO(weights)

    # TRAIN
    model.train(
        data=DATA_YAML,
        imgsz=imgsz,
        epochs=100,
        batch=16,
        device=0,
        project="train_runs",
        name=run_name,
        amp=True,
        patience=30,
        cache=True,

        # small-object-friendly augs
        degrees=5,
        translate=0.05,
        scale=0.15,
        mosaic=0.8,
        mixup=0.05,
        fliplr=0.5,
        hsv_h=0.015,
        hsv_s=0.5,
        hsv_v=0.2
    )

    # FIND BEST MODEL
    best_path = f"train_runs/{run_name}/weights/best.pt"
    save_path = f"{SAVE_DIR}/{run_name}.pt"
    os.system(f"cp {best_path} {save_path}")

    print(f"Saved: {save_path}")

    # EVALUATE ON YOUR DATA
    print(f"\nEvaluating {run_name}...\n")
    metrics = model.val(data=DATA_YAML, imgsz=imgsz, device=0)

    # STORE SUMMARY
    results_summary.append({
        "model": run_name,
        "imgsz": imgsz,
        "metrics": metrics.results_dict
    })


# ==============================================
# PRINT SUMMARY
# ==============================================
print("\n\n=========== FINAL SUMMARY ===========\n")
for r in results_summary:
    print(f"MODEL: {r['model']}  (imgsz={r['imgsz']})")
    print(r["metrics"])
    print("--------------------------------------")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

TRAINING: yolo11m_640  (imgsz=640)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/hornet_wasp_bee_dataset/data.yaml, degrees=5, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, h

In [ ]:
"""
train_all_variations.py

Train 8 YOLO11 variations:
  - yolo11{s,m,l,x} at imgsz 640 and 1024 (total 8 runs)

Requirements:
  - ultralytics
  - torch (CUDA-enabled on Colab A100)
  - Python 3.8+
  - Set DATA_YAML to your dataset yaml path
  - Large disk + GPU recommended

Notes:
  - This script explicitly disables Ultralytics' RandAugment (auto_augment),
    and avoids heavy CPU transforms that kill throughput.
  - If your dataset is on Google Drive, consider copying it to /content for best speed.
"""

import os
import shutil
import json
import time
from ultralytics import YOLO
import torch

# -----------------------------
# USER: update this path to your dataset YAML
# -----------------------------
DATA_YAML = "/content/drive/MyDrive/hornet_wasp_bee_dataset/data.yaml"  # change if needed
SAVE_DIR = "/content/trained_models_yolo11_variants"  # where final .pt and summary will be saved
os.makedirs(SAVE_DIR, exist_ok=True)

# -----------------------------
# device selection: try GPU, fallback to CPU
# -----------------------------
if torch.cuda.is_available():
    DEVICE = 0  # pass as device=0 to ultralytics
    print("[INFO] CUDA available - training on GPU.")
else:
    DEVICE = "cpu"
    print("[INFO] CUDA not found - training on CPU (slow).")

# -----------------------------
# base model weights (Ultralytics will download if needed)
# -----------------------------
BASE_WEIGHTS = {
    "s": "yolo11s.pt",
    "m": "yolo11m.pt",
    "l": "yolo11l.pt",
    "x": "yolo11x.pt",
}

# -----------------------------
# common template
# -----------------------------
HP_TEMPLATE = {
    "project": "runs_training_yolo11",
    "amp": True,
    "patience": 60,
    "cache": True,        # set to 'disk' if you want deterministic caching on Colab
    "save_period": 25,
    "close_mosaic": 20,
    "multi_scale": True,
    "workers": 8,
}

# -----------------------------
# Per-model/per-imgsz settings
# -----------------------------
RUN_CONFIGS = [
    # s
    ("s", 640, "yolo11s_640", {
        "epochs": 250, "batch": 16, "lr": 0.01,
        "degrees": 0.0, "translate": 0.02, "scale": 0.05,
        "mosaic": 0.20, "mixup": 0.0, "fliplr": 0.30,
        "hsv_h": 0.005, "hsv_s": 0.25, "hsv_v": 0.15,
    }),
    ("s", 1024, "yolo11s_1024", {
        "epochs": 250, "batch": 8, "lr": 0.006,
        "degrees": 0.0, "translate": 0.02, "scale": 0.04,
        "mosaic": 0.12, "mixup": 0.0, "fliplr": 0.25,
        "hsv_h": 0.005, "hsv_s": 0.20, "hsv_v": 0.12,
    }),

    # m
    ("m", 640, "yolo11m_640", {
        "epochs": 220, "batch": 12, "lr": 0.01,
        "degrees": 0.0, "translate": 0.03, "scale": 0.08,
        "mosaic": 0.25, "mixup": 0.02, "fliplr": 0.40,
        "hsv_h": 0.008, "hsv_s": 0.30, "hsv_v": 0.18,
    }),
    ("m", 1024, "yolo11m_1024", {
        "epochs": 220, "batch": 8, "lr": 0.006,
        "degrees": 0.0, "translate": 0.02, "scale": 0.06,
        "mosaic": 0.18, "mixup": 0.01, "fliplr": 0.35,
        "hsv_h": 0.006, "hsv_s": 0.25, "hsv_v": 0.15,
    }),

    # l
    ("l", 640, "yolo11l_640", {
        "epochs": 180, "batch": 8, "lr": 0.008,
        "degrees": 0.0, "translate": 0.04, "scale": 0.12,
        "mosaic": 0.28, "mixup": 0.03, "fliplr": 0.40,
        "hsv_h": 0.010, "hsv_s": 0.35, "hsv_v": 0.20,
        "weight_decay": 0.0005,
    }),
    ("l", 1024, "yolo11l_1024", {
        "epochs": 180, "batch": 6, "lr": 0.005,
        "degrees": 0.0, "translate": 0.03, "scale": 0.10,
        "mosaic": 0.20, "mixup": 0.02, "fliplr": 0.35,
        "hsv_h": 0.008, "hsv_s": 0.30, "hsv_v": 0.18,
        "weight_decay": 0.0006,
    }),

    # x
    ("x", 640, "yolo11x_640", {
        "epochs": 150, "batch": 6, "lr": 0.006,
        "degrees": 0.0, "translate": 0.04, "scale": 0.15,
        "mosaic": 0.30, "mixup": 0.04, "fliplr": 0.45,
        "hsv_h": 0.010, "hsv_s": 0.36, "hsv_v": 0.20,
        "weight_decay": 0.0006,
    }),
    ("x", 1024, "yolo11x_1024", {
        "epochs": 150, "batch": 4, "lr": 0.004,
        "degrees": 0.0, "translate": 0.03, "scale": 0.12,
        "mosaic": 0.22, "mixup": 0.02, "fliplr": 0.40,
        "hsv_h": 0.008, "hsv_s": 0.32, "hsv_v": 0.18,
        "weight_decay": 0.0007,
    }),
]

# -----------------------------
# Helper function to run one training job
# -----------------------------
def train_one(run_cfg):
    backbone_key, imgsz, run_name, params = run_cfg
    weights = BASE_WEIGHTS[backbone_key]
    print("\n" + "="*70)
    print(f"Starting run: {run_name}  (weights={weights}, imgsz={imgsz})")
    print("="*70 + "\n")

    # merge common settings and per-run params
    settings = dict(HP_TEMPLATE)
    settings.update(params)

    # device (ultralytics accepts device param)
    settings_device = DEVICE

    # prepare model
    model = YOLO(weights)

    # prepare argument dict for model.train
    # Note: we explicitly set auto_augment="none" to avoid RandAugment
    train_kwargs = {
        "data": DATA_YAML,
        "imgsz": imgsz,
        "epochs": settings["epochs"],
        "batch": settings["batch"],
        "device": settings_device,
        "project": settings["project"],
        "name": run_name,
        "amp": settings["amp"],
        "patience": settings["patience"],
        "cache": settings["cache"],
        "save_period": settings["save_period"],
        "multi_scale": settings.get("multi_scale", True),
        "workers": settings.get("workers", 8),

        # augmentations (tuned per-run)
        "degrees": settings.get("degrees", 0.0),
        "translate": settings.get("translate", 0.02),
        "scale": settings.get("scale", 0.05),
        "mosaic": settings.get("mosaic", 0.2),
        "mixup": settings.get("mixup", 0.0),
        "fliplr": settings.get("fliplr", 0.5),
        "hsv_h": settings.get("hsv_h", 0.005),
        "hsv_s": settings.get("hsv_s", 0.3),
        "hsv_v": settings.get("hsv_v", 0.2),

        # explicitly disable heavy/higher-cost augment pipelines
        "auto_augment": "none",   # IMPORTANT: disable randaugment
        "cutmix": 0.0,
        "copy_paste": 0.0,
        "erasing": 0.0,

        # regularization
        "weight_decay": settings.get("weight_decay", None),
        "close_mosaic": settings.get("close_mosaic", 20),
    }

    # remove None values to avoid API errors
    train_kwargs = {k: v for k, v in train_kwargs.items() if v is not None}

    # print run config summary
    print("Train kwargs summary:")
    for k, v in train_kwargs.items():
        print(f"  {k}: {v}")
    print("")

    # run training
    start = time.time()
    model.train(**train_kwargs)
    elapsed = time.time() - start
    print(f"[INFO] Training for {run_name} finished in {elapsed/60:.1f} minutes.")

    # copy best weights to SAVE_DIR
    best_path = os.path.join(settings["project"], run_name, "weights", "best.pt")
    if os.path.exists(best_path):
        dst = os.path.join(SAVE_DIR, f"{run_name}.pt")
        shutil.copyfile(best_path, dst)
        print(f"[INFO] Copied best model to {dst}")
    else:
        print(f"[WARN] Best weights not found at {best_path} (training may not have saved best).")

    # evaluate on val set
    print(f"[INFO] Running validation for {run_name} ...")
    results = model.val(data=DATA_YAML, imgsz=imgsz, device=settings_device)
    metrics = getattr(results, "results_dict", None) or {}
    print(f"[INFO] Validation metrics: {metrics}")

    return {
        "run_name": run_name,
        "weights": weights,
        "imgsz": imgsz,
        "train_kwargs": train_kwargs,
        "metrics": metrics
    }

# -----------------------------
# Run all configs sequentially and collect summaries
# -----------------------------
def main():
    summary = []
    for cfg in RUN_CONFIGS:
        res = train_one(cfg)
        summary.append(res)
        # small delay to flush resources if needed
        time.sleep(3)

    # save summary
    summary_path = os.path.join(SAVE_DIR, "results_summary.json")
    with open(summary_path, "w") as f:
        json.dump(summary, f, indent=2)
    print(f"[INFO] Saved summary to {summary_path}")

if __name__ == "__main__":
    main()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[INFO] CUDA available - training on GPU.

Starting run: yolo11s_640  (weights=yolo11s.pt, imgsz=640)

Train kwargs summary:
  data: /content/drive/MyDrive/hornet_wasp_bee_dataset/data.yaml
  imgsz: 640
  epochs: 250
  batch: 16
  device: 0
  project: runs_training_yolo11
  name: yolo11s_640
  amp: True
  patience: 60
  cache: True
  save_period: 25
  multi_scale: True
  workers: 8
  degrees: 0.0
  translate: 0.02
  scale: 0.05
  mosaic: 0.2
  mixup: 0.0
  fliplr: 0.3
  hsv_h: 0.005
  hsv_s: 0.25
  hsv_v: 0.15
  auto_augment: none
  cutmix: 0.0
  copy_paste: 0.0
  erasing: 0.0
  close_mosaic: 20

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 

KeyboardInterrupt: 

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

import os, shutil, random, glob
random.seed(42)

ROOT = "/content/drive/MyDrive/hornet_wasp_bee_dataset"

# Folders
train_img = f"{ROOT}/train/images"
train_lbl = f"{ROOT}/train/labels"
val_img   = f"{ROOT}/valid/images"
val_lbl   = f"{ROOT}/valid/labels"
test_img  = f"{ROOT}/test/images"
test_lbl  = f"{ROOT}/test/labels"

# Make sure folders exist
for d in [val_img, val_lbl, test_img, test_lbl]:
    os.makedirs(d, exist_ok=True)

# Helper to load image names
def list_images(folder):
    exts = ("*.jpg","*.jpeg","*.png","*.bmp")
    files = []
    for e in exts:
        files += glob.glob(os.path.join(folder, e))
    return sorted([os.path.basename(f) for f in files])

train_list = list_images(train_img)
val_list   = list_images(val_img)
test_list  = list_images(test_img)

print("Before:", len(train_list), "train |", len(val_list), "val |", len(test_list), "test")

total = len(train_list) + len(val_list) + len(test_list)
target_train = int(round(total * 0.80))
target_val   = int(round(total * 0.10))
target_test  = int(round(total * 0.10))

need_val  = target_val - len(val_list)
need_test = target_test - len(test_list)

print(f"Targets → Train={target_train}, Val={target_val}, Test={target_test}")
print(f"Need to move: {need_val} → valid, {need_test} → test")

# Build candidate list (only those with labels)
candidates = []
for img in train_list:
    label = img.rsplit(".",1)[0] + ".txt"
    if os.path.exists(f"{train_lbl}/{label}"):
        candidates.append(img)

random.shuffle(candidates)

to_val  = candidates[:need_val]
to_test = candidates[need_val:need_val+need_test]

def copy_pair(img_name, src_i, src_l, dst_i, dst_l):
    base = img_name.rsplit(".",1)[0]
    img_src = f"{src_i}/{img_name}"
    lbl_src = f"{src_l}/{base}.txt"
    img_dst = f"{dst_i}/{img_name}"
    lbl_dst = f"{dst_l}/{base}.txt"

    shutil.copy2(img_src, img_dst)
    shutil.copy2(lbl_src, lbl_dst)

print("Copying validation images...")
for x in to_val:
    copy_pair(x, train_img, train_lbl, val_img, val_lbl)

print("Copying test images...")
for x in to_test:
    copy_pair(x, train_img, train_lbl, test_img, test_lbl)

print("Done.\nFinal counts:")
print("Train:", len(list_images(train_img)))
print("Valid:", len(list_images(val_img)))
print("Test :", len(list_images(test_img)))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Before: 1150 train | 6 val | 3 test
Targets → Train=927, Val=116, Test=116
Need to move: 110 → valid, 113 → test
Copying validation images...
Copying test images...
Done.
Final counts:
Train: 1150
Valid: 116
Test : 116


In [ ]:
import os

ROOT = "/content/drive/MyDrive/hornet_wasp_bee_dataset"

train_img = f"{ROOT}/train/images"
train_lbl = f"{ROOT}/train/labels"
val_img   = f"{ROOT}/valid/images"
val_lbl   = f"{ROOT}/valid/labels"
test_img  = f"{ROOT}/test/images"
test_lbl  = f"{ROOT}/test/labels"

def delete_pair(img_name):
    base = img_name.rsplit(".",1)[0]
    img_path = f"{train_img}/{img_name}"
    lbl_path = f"{train_lbl}/{base}.txt"

    if os.path.exists(img_path):
        os.remove(img_path)
        print("Deleted image:", img_path)

    if os.path.exists(lbl_path):
        os.remove(lbl_path)
        print("Deleted label:", lbl_path)

# ===== DELETE VALID SET ORIGINALS =====
val_imgs_list = os.listdir(val_img)
print("\n[INFO] Deleting originals moved to VALID:")
for img in val_imgs_list:
    delete_pair(img)

# ===== DELETE TEST SET ORIGINALS =====
test_imgs_list = os.listdir(test_img)
print("\n[INFO] Deleting originals moved to TEST:")
for img in test_imgs_list:
    delete_pair(img)

print("\n[DONE] Train set cleaned.")



[INFO] Deleting originals moved to VALID:
Deleted image: /content/drive/MyDrive/hornet_wasp_bee_dataset/train/images/131265303_9767644739_n_47f4f6ac-89aa-4bb3-98b6-e000af18af49_jpeg.rf.b19056a9d5cf7766949ee6a2b2e012ab.jpg
Deleted label: /content/drive/MyDrive/hornet_wasp_bee_dataset/train/labels/131265303_9767644739_n_47f4f6ac-89aa-4bb3-98b6-e000af18af49_jpeg.rf.b19056a9d5cf7766949ee6a2b2e012ab.txt
Deleted image: /content/drive/MyDrive/hornet_wasp_bee_dataset/train/images/https3A2F2Finaturalist-open-data-s3-amazonaws-com2Fphotos2F1355249352Foriginal_56a5cc5d-8f76-4669-97a0-a6fdc54d8fb5_jpeg.rf.6456c9650e3b6f529160c18638d28319.jpg
Deleted label: /content/drive/MyDrive/hornet_wasp_bee_dataset/train/labels/https3A2F2Finaturalist-open-data-s3-amazonaws-com2Fphotos2F1355249352Foriginal_56a5cc5d-8f76-4669-97a0-a6fdc54d8fb5_jpeg.rf.6456c9650e3b6f529160c18638d28319.txt
Deleted image: /content/drive/MyDrive/hornet_wasp_bee_dataset/train/images/53209462_ba37d8745d_m_5b09fbf8-862f-40dd-88c5-1200

In [ ]:
print("Done.\nFinal counts:")
print("Train:", len(list_images(train_img)))
print("Valid:", len(list_images(val_img)))
print("Test :", len(list_images(test_img)))

Done.
Final counts:
Train: 927
Valid: 116
Test : 116


In [ ]:
"""
Train 8 YOLO11 variants (s, m, l, x @ 640 & 1024)
Optimized for small-object detection (bees/hornets/wasps)

This script:
  ✓ Detects GPU
  ✓ Copies dataset from Drive -> local disk for speed (if available)
  ✓ Trains each model with tuned hyperparameters
  ✓ Uses AdamW optimizer for faster convergence
  ✓ Saves best weights
  ✓ Validates and stores metrics
  ✓ Produces a final summary JSON

Run in Colab:
---------------
!pip install -U ultralytics
python train_yolo11_all_variants.py
---------------
"""

import os
import shutil
import json
import time
import torch
from ultralytics import YOLO

# ============================================================
# DATASET CONFIG (DRIVE -> LOCAL COPY)
# ============================================================

# Original dataset root on Drive
DRIVE_DATA_ROOT = "/content/drive/MyDrive/hornet_wasp_bee_dataset"

# Faster local copy (on Colab VM)
LOCAL_DATA_ROOT = "/content/hornet_wasp_bee_dataset"

def ensure_local_dataset():
    """
    If dataset exists on Drive and not yet copied locally,
    copy it to /content for much faster IO.
    """
    if os.path.exists(LOCAL_DATA_ROOT):
        print(f"[DATA] Using existing local dataset at: {LOCAL_DATA_ROOT}")
        return

    if not os.path.exists(DRIVE_DATA_ROOT):
        print("[WARN] Drive dataset path not found:", DRIVE_DATA_ROOT)
        print("[WARN] Make sure your dataset is mounted correctly.")
        return

    print(f"[DATA] Copying dataset from {DRIVE_DATA_ROOT} to {LOCAL_DATA_ROOT}...")
    shutil.copytree(DRIVE_DATA_ROOT, LOCAL_DATA_ROOT)
    print("[DATA] Copy complete. Training will now use local disk for speed.")

ensure_local_dataset()

# Final DATA_YAML path (prefer local, fall back to Drive)
if os.path.exists(os.path.join(LOCAL_DATA_ROOT, "data.yaml")):
    DATA_YAML = os.path.join(LOCAL_DATA_ROOT, "data.yaml")
else:
    DATA_YAML = os.path.join(DRIVE_DATA_ROOT, "data.yaml")

print("[DATA] Using data.yaml at:", DATA_YAML)

# Directory to store final trained weights + summary
SAVE_DIR = "/content/trained_yolo11_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# ============================================================
# DEVICE SELECTION
# ============================================================
if torch.cuda.is_available():
    DEVICE = 0
    print("[INFO] Using GPU")
else:
    DEVICE = "cpu"
    print("[INFO] Using CPU (slow!)")

# ============================================================
# BASE MODEL WEIGHTS (Ultralytics will auto-download)
# ============================================================
BASE_WEIGHTS = {
    "s": "yolo11s.pt",
    "m": "yolo11m.pt",
    "l": "yolo11l.pt",
    "x": "yolo11x.pt",
}

# ============================================================
# TUNED HYPERPARAMETERS FOR EACH VARIANT
# Switched optimizer -> AdamW
# multi_scale is disabled in train() for speed
# workers set to 2 (better for Colab CPUs)
# ============================================================

RUN_CONFIGS = [

    # ========================================================
    # YOLO11-SMALL (FASTEST, GOOD FOR REALTIME)
    # ========================================================
    ("s", 640, "yolo11s_640", {
        "epochs": 220,
        "batch": 16,
        "lr0": 0.0015,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.02,
        "scale": 0.05,
        "mosaic": 0.20,
        "mixup": 0.0,
        "fliplr": 0.30,
        "hsv_h": 0.005,
        "hsv_s": 0.25,
        "hsv_v": 0.15,
    }),

    ("s", 1024, "yolo11s_1024", {
        "epochs": 220,
        "batch": 8,
        "lr0": 0.0012,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.015,
        "scale": 0.03,
        "mosaic": 0.12,
        "mixup": 0.0,
        "fliplr": 0.25,
        "hsv_h": 0.005,
        "hsv_s": 0.22,
        "hsv_v": 0.12,
    }),

    # ========================================================
    # YOLO11-MEDIUM (BEST BALANCE)
    # ========================================================
    ("m", 640, "yolo11m_640", {
        "epochs": 200,
        "batch": 12,
        "lr0": 0.0013,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.03,
        "scale": 0.08,
        "mosaic": 0.22,
        "mixup": 0.02,
        "fliplr": 0.40,
        "hsv_h": 0.008,
        "hsv_s": 0.30,
        "hsv_v": 0.18,
    }),

    ("m", 1024, "yolo11m_1024", {
        "epochs": 200,
        "batch": 8,
        "lr0": 0.0010,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.02,
        "scale": 0.06,
        "mosaic": 0.18,
        "mixup": 0.01,
        "fliplr": 0.35,
        "hsv_h": 0.006,
        "hsv_s": 0.26,
        "hsv_v": 0.15,
    }),

    # ========================================================
    # YOLO11-LARGE
    # ========================================================
    ("l", 640, "yolo11l_640", {
        "epochs": 180,
        "batch": 8,
        "lr0": 0.0010,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.035,
        "scale": 0.12,
        "mosaic": 0.25,
        "mixup": 0.03,
        "fliplr": 0.40,
        "hsv_h": 0.009,
        "hsv_s": 0.33,
        "hsv_v": 0.20,
        "weight_decay": 0.0005,
    }),

    ("l", 1024, "yolo11l_1024", {
        "epochs": 180,
        "batch": 6,
        "lr0": 0.0009,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.03,
        "scale": 0.10,
        "mosaic": 0.18,
        "mixup": 0.02,
        "fliplr": 0.35,
        "hsv_h": 0.008,
        "hsv_s": 0.30,
        "hsv_v": 0.18,
        "weight_decay": 0.0006,
    }),

    # ========================================================
    # YOLO11-X (MOST POWERFUL)
    # ========================================================
    ("x", 640, "yolo11x_640", {
        "epochs": 150,
        "batch": 6,
        "lr0": 0.0009,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.035,
        "scale": 0.14,
        "mosaic": 0.26,
        "mixup": 0.04,
        "fliplr": 0.45,
        "hsv_h": 0.009,
        "hsv_s": 0.36,
        "hsv_v": 0.20,
        "weight_decay": 0.0006,
    }),

    ("x", 1024, "yolo11x_1024", {
        "epochs": 150,
        "batch": 4,
        "lr0": 0.0008,
        "optimizer": "AdamW",
        "degrees": 0,
        "translate": 0.03,
        "scale": 0.12,
        "mosaic": 0.20,
        "mixup": 0.03,
        "fliplr": 0.40,
        "hsv_h": 0.008,
        "hsv_s": 0.32,
        "hsv_v": 0.18,
        "weight_decay": 0.0007,
    }),
]

# ============================================================
# TRAIN ONE MODEL
# ============================================================

def train_one(backbone, imgsz, run_name, cfg):
    weights = BASE_WEIGHTS[backbone]

    print("\n" + "=" * 70)
    print(f"Starting: {run_name}  (model={weights}, imgsz={imgsz})")
    print("=" * 70)

    model = YOLO(weights)

    train_args = {
        "data": DATA_YAML,
        "imgsz": imgsz,
        "epochs": cfg["epochs"],
        "batch": cfg["batch"],
        "device": DEVICE,
        "project": "runs_yolo11",
        "name": run_name,
        "lr0": cfg["lr0"],
        "optimizer": cfg["optimizer"],
        "weight_decay": cfg.get("weight_decay", 0.0005),
        "patience": 60,

        # key speed tweak: multi_scale off
        "multi_scale": False,

        # cache images on first epoch then read from disk
        "cache": "disk",

        # less frequent saving
        "save_period": 25,
        "close_mosaic": 20,
        "plots": False,

        # dataloader workers (Colab-friendly)
        "workers": 2,

        # augmentations
        "degrees": cfg["degrees"],
        "translate": cfg["translate"],
        "scale": cfg["scale"],
        "mosaic": cfg["mosaic"],
        "mixup": cfg["mixup"],
        "fliplr": cfg["fliplr"],
        "hsv_h": cfg["hsv_h"],
        "hsv_s": cfg["hsv_s"],
        "hsv_v": cfg["hsv_v"],

        # disable heavy augs
        "auto_augment": "none",
        "cutmix": 0.0,
        "copy_paste": 0.0,
        "erasing": 0.0,
    }

    # TRAIN
    model.train(**train_args)

    # SAVE BEST WEIGHTS
    best_path = f"runs_yolo11/{run_name}/weights/best.pt"
    final_path = f"{SAVE_DIR}/{run_name}.pt"
    if os.path.exists(best_path):
        shutil.copy(best_path, final_path)
        print(f"[SAVED] {final_path}")
    else:
        print(f"[WARN] best.pt not found for {run_name}")

    # VALIDATE
    print("[INFO] Validating...")
    metrics = model.val(data=DATA_YAML, imgsz=imgsz, device=DEVICE)
    return metrics.results_dict if hasattr(metrics, "results_dict") else {}

# ============================================================
# MAIN LOOP
# ============================================================

def main():
    results_summary = []

    for backbone, imgsz, name, cfg in RUN_CONFIGS:
        metrics = train_one(backbone, imgsz, name, cfg)
        results_summary.append({
            "model": name,
            "imgsz": imgsz,
            "metrics": metrics
        })

    summary_path = f"{SAVE_DIR}/training_summary.json"
    with open(summary_path, "w") as f:
        json.dump(results_summary, f, indent=2)

    print("\n======== TRAINING COMPLETE ========")
    print("Summary saved to:", summary_path)
    print("All models saved to:", SAVE_DIR)

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    173/220      4.89G     0.3734       0.35     0.9193         18        640: 100% ━━━━━━━━━━━━ 58/58 9.1it/s 6.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 9.1it/s 0.4s
                   all        116        136      0.937      0.785      0.892      0.803

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    174/220      4.89G     0.3592     0.3223     0.9005         18        640: 100% ━━━━━━━━━━━━ 58/58 9.1it/s 6.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 8.8it/s 0.5s
                   all        116        136      0.925      0.801      0.896      0.795

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    175/220      4.89G     0.3563      0.322     0.90

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x783d305ce160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^

     70/180      10.7G      1.056      1.019      1.707          7       1024: 81% ━━━━━━━━━╸── 126/155 5.6it/s 22.6s<5.1s

^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x783d305ce160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process


     70/180      10.7G      1.054      1.014      1.707          6       1024: 85% ━━━━━━━━━━── 132/155 5.2it/s 23.8s<4.4s

...

     70/150      7.88G      1.123      1.164      1.481          3        640: 100% ━━━━━━━━━━━━ 155/155 6.3it/s 24.7s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 11.3it/s 0.9s
                   all        116        136      0.819      0.551      0.686      0.461

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     71/150      7.88G      1.027      1.063      1.404          3        640: 100% ━━━━━━━━━━━━ 155/155 6.2it/s 24.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 11.4it/s 0.9s
                   all        116        136      0.728      0.629      0.691      0.474

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     72/150      7.88G      1.053       1.08      1.414          4        640: 100% ━━━━━━━━━━━━ 155/155 6.3it/s 24.6s
                 Class     Images  Instances      B

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x783d305ce160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x783d305ce160>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

     41/150      11.9G      1.757      2.067      2.407         11       1024: 26% ━━━───────── 61/232 5.6it/s 11.0s<30.7s

if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^Exception ignored in: ^^<function _MultiProcessingDataLoaderIter.__del__ at 0x783d305ce160>^
^Traceback (most recent call last):
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
^^    ^self._shutdown_workers()^
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^    ^if w.is_alive():^
^^   ^  ^  ^^^^^^^^
^AssertionError^: ^can only test a child process^
^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter._

     41/150      11.9G       1.71      2.075       2.36          5       1024: 100% ━━━━━━━━━━━━ 232/232 5.7it/s 40.8s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 15/15 8.9it/s 1.7s
                   all        116        136      0.503      0.346      0.386        0.2

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     42/150      11.9G      1.744      2.042      2.388          3       1024: 100% ━━━━━━━━━━━━ 232/232 5.7it/s 40.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 15/15 9.0it/s 1.7s
                   all        116        136      0.448      0.384      0.356      0.171

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     43/150      11.9G      1.711      2.005      2.361          4       1024: 100% ━━━━━━━━━━━━ 232/232 5.8it/s 40.2s
                 Class     Images  Instances      Box

In [ ]:
!ls /content/trained_yolo11_models

In [ ]:
# Copy trained models + summary from session to Google Drive
import shutil
import os

SRC = "/content/trained_yolo11_models"
DST = "/content/drive/MyDrive/trained_yolo11_models_backup"

if not os.path.exists(SRC):
    print("Source folder not found. Did training finish?")
else:
    shutil.copytree(SRC, DST, dirs_exist_ok=True)
    print("✔ All trained models copied to Google Drive at:")
    print(DST)


✔ All trained models copied to Google Drive at:
/content/drive/MyDrive/trained_yolo11_models_backup


In [4]:
import os
import shutil
import json
import torch
from ultralytics import YOLO

# ============================================================
# CONFIGURATION
# ============================================================

# Original dataset root on Drive (Directly used for data.yaml and caching)
DRIVE_DATA_ROOT = "/content/drive/MyDrive/hornet_wasp_bee_dataset"
# Directory to store final trained weights + summary
SAVE_DIR = "/content/trained_yolov8l_model"
os.makedirs(SAVE_DIR, exist_ok=True)

# Final DATA_YAML path (Assumes data.yaml is at the root of the dataset)
DATA_YAML = os.path.join(DRIVE_DATA_ROOT, "data.yaml")

# Device Selection
DEVICE = 0 if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using {'GPU' if DEVICE == 0 else 'CPU (slow!)'}")
print("[DATA] Using data.yaml directly from Drive for caching:", DATA_YAML)

# ============================================================
# YOLOv8L TRAINING CONFIG (Tuned for small-object/high-res)
# ============================================================

YOLOV8L_CONFIG = {
    "model_name": "yolov8l.pt",  # YOLOv8 large model weights
    "imgsz": 1024,               # Use high resolution for small objects
    "run_name": "yolov8l_1024_small_object_disk_cache",
    "cfg": {
        # Adapted for 'l' model (fewer epochs, lower batch/lr, higher augmentations)
        "epochs": 180,
        "batch": 6,              # Reduced batch size for YOLOv8l @ 1024px on typical Colab GPU
        "lr0": 0.0009,
        "optimizer": "AdamW",
        "weight_decay": 0.0006,
        "patience": 60,

        # Augmentations (Higher values for 'l' model for more regularization)
        "degrees": 0,
        "translate": 0.03,
        "scale": 0.10,
        "mosaic": 0.20,
        "mixup": 0.02,
        "fliplr": 0.40,
        "hsv_h": 0.008,
        "hsv_s": 0.32,
        "hsv_v": 0.18,
    }
}


def train_one(model_name, imgsz, run_name, cfg):
    print("\n" + "=" * 70)
    print(f"Starting: {run_name}  (model={model_name}, imgsz={imgsz})")
    print("=" * 70)

    # Load YOLOv8 model
    model = YOLO(model_name)

    train_args = {
        "data": DATA_YAML,
        "imgsz": imgsz,
        "device": DEVICE,
        "project": "runs_yolov8l",
        "name": run_name,
        "workers": 2,            # Colab-friendly

        # KEY CHANGE: Cache to disk on first epoch (reads from Drive, writes to local disk)
        "cache": "disk",

        # Hyperparameters
        "epochs": cfg["epochs"],
        "batch": cfg["batch"],
        "lr0": cfg["lr0"],
        "optimizer": cfg["optimizer"],
        "weight_decay": cfg["weight_decay"],
        "patience": cfg["patience"],

        # Augmentations
        "degrees": cfg["degrees"],
        "translate": cfg["translate"],
        "scale": cfg["scale"],
        "mosaic": cfg["mosaic"],
        "mixup": cfg["mixup"],
        "fliplr": cfg["fliplr"],
        "hsv_h": cfg["hsv_h"],
        "hsv_s": cfg["hsv_s"],
        "hsv_v": cfg["hsv_v"],

        "close_mosaic": 20,
        "plots": False,
    }

    # TRAIN
    results = model.train(**train_args)

    # SAVE BEST WEIGHTS
    best_path = os.path.join(results.save_dir, "weights", "best.pt")
    final_path = os.path.join(SAVE_DIR, f"{run_name}.pt")
    if os.path.exists(best_path):
        shutil.copy(best_path, final_path)
        print(f"[SAVED] {final_path}")
    else:
        print(f"[WARN] best.pt not found for {run_name}")

    # VALIDATE AND RETURN METRICS
    print("[INFO] Validating...")
    metrics = model.val(data=DATA_YAML, imgsz=imgsz, device=DEVICE)
    return metrics.results_dict if hasattr(metrics, "results_dict") else {}


# ============================================================
# MAIN EXECUTION
# ============================================================
def main():
    config = YOLOV8L_CONFIG
    metrics = train_one(config["model_name"], config["imgsz"], config["run_name"], config["cfg"])

    results_summary = [{
        "model": config["run_name"],
        "imgsz": config["imgsz"],
        "metrics": metrics
    }]

    summary_path = os.path.join(SAVE_DIR, "yolov8l_training_summary.json")
    with open(summary_path, "w") as f:
        json.dump(results_summary, f, indent=2)

    print("\n======== TRAINING COMPLETE ========")
    print("Summary saved to:", summary_path)
    print("Model saved to:", SAVE_DIR)


if __name__ == "__main__":
    main()

[DATA] Found 3364 items to copy.


[DATA] Copying Dataset: 100%|██████████| 3364/3364 [00:46<00:00, 71.82file/s]


[DATA] Copy complete. Training will now use local disk for speed.
[INFO] Using GPU
[DATA] Using data.yaml at: /content/hornet_wasp_bee_dataset/data.yaml

Starting: yolov8l_1024_small_object_local_disk  (model=yolov8l.pt, imgsz=1024)
Ultralytics 8.3.236 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/hornet_wasp_bee_dataset/data.yaml, degrees=0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=180, erasing=0.4, exist_ok=False, fliplr=0.4, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.008, hsv_s=0.32, hsv_v=0.18, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0009, lrf=0.01, 

KeyboardInterrupt: 

In [2]:
!pip install ultralytics==8.3.24

from google.colab import drive
drive.mount('/content/drive')

from ultralytics import YOLO
import torch
print("GPU:", torch.cuda.get_device_name(0))

model = YOLO("yolo11l.pt")  # YOLOv11-L

results = model.train(
    data="/content/hornet_wasp_bee_dataset/data.yaml",  # change this
    epochs=100,
    imgsz=1024,
    batch=16,
    device=0,

    # === Strong YOLOv8-style hyperparameters ===
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    box=7.5,
    cls=0.5,
    dfl=1.5,

    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    flipud=0.0,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,
    auto_augment="randaugment",
    erasing=0.4,

    close_mosaic=10,
    dropout=0.0,

    pretrained=True,
    optimizer="auto",
    deterministic=True,
    val=True,
)

from IPython.display import Image
Image(filename='runs/detect/train/results.png')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: NVIDIA A100-SXM4-80GB
New https://pypi.org/project/ultralytics/8.3.236 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.24 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/content/hornet_wasp_bee_dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=Fal

train: Scanning /content/hornet_wasp_bee_dataset/train/labels.cache... 927 images, 19 backgrounds, 0 corrupt: 100%|██████████| 927/927 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



/usr/local/lib/python3.12/dist-packages/ultralytics/data/augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning /content/hornet_wasp_bee_dataset/valid/labels.cache... 116 images, 4 backgrounds, 0 corrupt: 100%|██████████| 116/116 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        116        136       0.69      0.694      0.717      0.456



100 epochs completed in 0.663 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 51.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 51.3MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.24 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLO11l summary (fused): 464 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.32it/s]


                   all        116        136      0.731       0.65      0.716      0.461
          Asian Hornet         53         53      0.738      0.868      0.879      0.548
                  Bees         18         29      0.889       0.55       0.73      0.448
                Hornet         22         22      0.535      0.682      0.683      0.479
                  Wasp         19         32      0.762        0.5      0.573      0.368
Speed: 0.3ms preprocess, 6.4ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train3


FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/train/results.png'